In [22]:
%%shell
pip install datasets
pip install -U sentence-transformers 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 15.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!tar -xf /content/gdrive/MyDrive/data/hasan-etal-2020-low.tar.bz2

In [12]:
with open('/content/hasan-etal-2020-low/2.75M/original_corpus.bn', 'r') as f:
    all_bangla_text = f.read()

In [2]:
all_b_text = all_bangla_text.split("\n")

In [3]:
with open('/content/hasan-etal-2020-low/2.75M/original_corpus.en', 'r') as f:
    all_english_text = f.read()

In [4]:
all_e_text = all_english_text.split("\n")

In [5]:
len(all_e_text)

2753070

In [14]:
e_text = all_e_text[:300000]
b_text = all_b_text[:300000]

len(e_text) , len(b_text)

(300000, 300000)

In [15]:
train_samples = []
for m , n in zip(e_text,b_text):
  train_samples.append(m+'\t'+n)

In [16]:
len(train_samples)

300000

In [25]:
import pickle

with open('e2b3lakh.bin','wb') as f:
  pickle.dump(train_samples,f)

In [17]:
train_samples[0]

'He turned to look Langdon in the eye.\tসে ঘুরে ল্যাংডনের চোখের দিকে তাকায়।'

In [20]:
import gzip
import os

if not os.path.exists('./data'):
    os.mkdir('./data')

# save to file, sentence transformers reader will expect tsv.gz file
with gzip.open(f'./data/e2b3lakh.tsv.gz', 'wt', encoding='utf-8') as f:
      f.write('\n'.join(train_samples))

In [23]:
from sentence_transformers import models, SentenceTransformer

xlmr = models.Transformer('xlm-roberta-base')
pooler = models.Pooling(
    xlmr.get_word_embedding_dimension(),
    pooling_mode_mean_tokens=True
)

student = SentenceTransformer(modules=[xlmr, pooler])
student

Downloading:   0%|          | 0.00/615 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [24]:
teacher = SentenceTransformer('paraphrase-distilroberta-base-v2')
# teacher = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L6-v2')

teacher

Downloading:   0%|          | 0.00/736 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/686 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: RobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [26]:
batch_size = 32

from sentence_transformers import ParallelSentencesDataset

data = ParallelSentencesDataset(student_model=student, 
                                teacher_model=teacher,
                                batch_size=batch_size, 
                                use_embedding_cache=True)


In [28]:
max_sentence_length = 256
data.load_data('/content/data/e2b3lakh.tsv.gz', max_sentence_length=max_sentence_length)

In [29]:
from torch.utils.data import DataLoader

loader = DataLoader(data, shuffle=True, batch_size=batch_size)

In [30]:
from sentence_transformers import losses

loss = losses.MSELoss(model=student)

In [31]:
out_dir = "/content/gdrive/MyDrive/roberta-st"

In [ ]:
from sentence_transformers import evaluation
import numpy as np

epochs = 7
warmup_steps = int(len(loader) * epochs * 0.1)

student.fit(
    train_objectives=[(loader, loss)],
    epochs=epochs,
    warmup_steps=warmup_steps,
    output_path=out_dir,
    optimizer_params={'lr': 2e-5, 'eps': 1e-6}, #'correct_bias': False},
    save_best_model=True,
    show_progress_bar=True
)

Epoch:   0%|          | 0/7 [00:00<?, ?it/s]

Iteration:   0%|          | 0/15718 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/sentence_transformers/SentenceTransformer.py:547: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:230.)
  labels = torch.tensor(labels)
